## Import packages

In [30]:
import pandas as pd
import numpy as np
import os

## Implementation

### Read data

In [31]:
path = 'YOUR FILE PATH'
file_name = 'YOUR FILE NAME'

input_file = os.path.join(path, file_name)
df = pd.read_csv(input_file)

In [32]:
df.columns = ['stock_id', 'index_num', 'date', 'open', 'close']
# df.set_index('stock_id', inplace=True)
df.drop(columns=['index_num'], inplace=True)
df['returns'] = df['close']/df['open']-1
df.head()

,stock_id,date,open,close,returns
0,688089.XSHG,2021-11-24 14:25:00,53.55,53.78,0.004295
1,688089.XSHG,2021-11-24 14:30:00,53.75,53.73,-0.000372
2,688089.XSHG,2021-11-24 14:35:00,53.73,53.55,-0.003350
3,688089.XSHG,2021-11-24 14:40:00,53.70,53.61,-0.001676
4,688089.XSHG,2021-11-24 14:45:00,53.62,53.67,0.000932


### Prepare the dataset

In [33]:
stock_list = df['stock_id'].unique().tolist()
keep_cols = ['date', 'returns']

data = df[['date']].drop_duplicates()
for i in range(len(stock_list)):
    df1 = df[df['stock_id']==stock_list[i]][keep_cols]
    df1.columns = ['date', stock_list[i]]
    data = pd.merge(data, df1, on='date', how='left')

# Prepare the dataset for the computing process
data = data[0:5000]
data.dropna(inplace=True, axis=1)
data.set_index('date', inplace=True)

### Compute the optimal weights

In [34]:
# index_row = data.index
# index_col = data.columns

# data_T = pd.DataFrame(data.values.T, columns=index_row, index=index_col)
# Sigma = data_T.corr()
# Sigma_inv = pd.DataFrame(np.linalg.pinv(Sigma.values), columns=Sigma.columns, index=Sigma.index)

# test_inv = Sigma_inv.dot(Sigma)

# Calculate the covariance matrix of stock returns
Sigma = data.corr()
Sigma_inv = pd.DataFrame(np.linalg.pinv(Sigma.values), columns=Sigma.columns, index=Sigma.index).to_numpy()

In [35]:
# Get the optimal weight for each stock
Identity = pd.DataFrame(np.ones([12,1]))
Identity_T = pd.DataFrame(np.ones([1,12]))
a = (Identity_T.dot(Sigma_inv)).dot(Identity).values
weight_opt = Sigma_inv.dot(Identity) / a

print(weight_opt)
print('Sum of weights: ', weight_opt.sum())

[[0.11851668]
 [0.09194967]
 [0.07991533]
 [0.1574547 ]
 [0.04329583]
 [0.06184514]
 [0.06437801]
 [0.06400807]
 [0.09100592]
 [0.08134872]
 [0.06557228]
 [0.08070965]]
Sum of weights:  1.0
